<h3>Model</h3>

In [25]:
# python standard libraries
import os
import random
import fnmatch
import datetime
import pickle
import re

# data processing
import numpy as np
np.set_printoptions(formatter={'float_kind':lambda x: "%.4f" % x})

import pandas as pd
pd.set_option('display.width', 300)
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_colwidth', 200)

# tensorflow
import tensorflow as tf
import keras
from keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.models import load_model

# sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# imaging
import cv2
from imgaug import augmenters as img_aug
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image

In [3]:
data_dir = './Data'
data_folders = os.listdir(data_dir)
file_list = []
for dir in data_folders:
    file_list = file_list + os.listdir(os.path.join(data_dir,dir))
print(file_list)

['test0.png', 'test1.png', 'test10.png', 'test100.png', 'test101.png', 'test102.png', 'test103.png', 'test104.png', 'test105.png', 'test106.png', 'test107.png', 'test108.png', 'test109.png', 'test11.png', 'test110.png', 'test111.png', 'test112.png', 'test113.png', 'test114.png', 'test115.png', 'test116.png', 'test117.png', 'test118.png', 'test119.png', 'test12.png', 'test120.png', 'test121.png', 'test122.png', 'test123.png', 'test124.png', 'test125.png', 'test126.png', 'test127.png', 'test128.png', 'test129.png', 'test13.png', 'test130.png', 'test131.png', 'test132.png', 'test133.png', 'test134.png', 'test135.png', 'test136.png', 'test137.png', 'test138.png', 'test139.png', 'test14.png', 'test140.png', 'test141.png', 'test142.png', 'test143.png', 'test144.png', 'test145.png', 'test146.png', 'test147.png', 'test148.png', 'test149.png', 'test15.png', 'test150.png', 'test151.png', 'test152.png', 'test153.png', 'test154.png', 'test155.png', 'test156.png', 'test16.png', 'test17.png', 'test1

In [21]:
data_dir = './Data'
data_folders = os.listdir(data_dir)
file_list = []
for dir in data_folders:
    file_list = file_list + os.listdir(os.path.join(data_dir,dir))
    
image_paths = []
steering_direction = []
pattern = "*.png"
for filename in file_list:
    if fnmatch.fnmatch(filename, pattern):
        image_paths.append(os.path.join(data_dir,filename))
        angle = re.sub('.png','',re.sub('.*dir_','',filename))  # 092 part of video01_143_092.png is the angle. 90 is go straight
        steering_direction.append(angle)

The size of the validation set is not a good split. But we want to train the model on as much data as possible.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split( image_paths, steering_direction, test_size=0.1)
print("Training data: %d\nValidation data: %d" % (len(X_train), len(X_valid)))

In [ ]:
def zoom(image):
    zoom = img_aug.Affine(scale=(1.1, 1.3))  # zoom from 100% (no zoom) to 130%
    image = zoom.augment_image(image)
    return image

In [ ]:
def flip(image, steering_direction):
    image = cv2.flip(image,1)
    steering_direction = -steering_direction
    return image, steering_direction

In [ ]:
def random_augment(image, steering_angle):
    # if np.random.rand() < 0.5:
    #     image = zoom(image)
    # if np.random.rand() < 0.5:
    #     image = adjust_brightness(image)
    # image, steering_angle = random_flip(image, steering_angle)
    
    return image, steering_angle

Image processing is necesary to use the images with the nvidia model.

In [ ]:
def img_preprocess(image):
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relevant for lane following
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (200,66)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing the values of the image. We dont need to normalize it in the model anny more
    return image

In [24]:
def nvidia_model():
    model = Sequential(name='Nvidia_Model')
    
    # elu=Expenential Linear Unit, similar to leaky Relu
    # skipping 1st hiddel layer (nomralization layer), as we have normalized the data
    
    # Convolution Layers
    model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu')) 
    model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    
    # output layer: turn angle (from 45-135, 90 is straight, <90 turn left, >90 turn right)
    model.add(Dense(activation="softmax", units=3))
    
    # since this is a regression problem not classification problem
    model.compile(optimizer = 'adam', loss = 'categorical_hinge', metrics = ['accuracy'])
    
    return model

model = nvidia_model()
print(model.summary())

ValueError: Unknown activation function: asdf. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
def image_data_generator(image_paths, steering_angles, batch_size, is_training):
    while True:
        batch_images = []
        batch_steering_angles = []
        
        for i in range(batch_size):
            random_index = random.randint(0, len(image_paths) - 1)
            image_path = image_paths[random_index]
            image = cv2.imread(image_paths[random_index])
            steering_angle = steering_angles[random_index]
            if is_training:
                # training: augment image
                image, steering_angle = random_augment(image, steering_angle)
              
            image = img_preprocess(image)
            batch_images.append(image)
            batch_steering_angles.append(steering_angle)
            
        yield( np.asarray(batch_images), np.asarray(batch_steering_angles))

In [ ]:
model_output_dir = "./Model"


# saves the model weights after each epoch if the validation loss decreased
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(model_output_dir,'lane_navigation_check.h5'), verbose=1, save_best_only=True)

history = model.fit_generator(image_data_generator( X_train, y_train, batch_size=100, is_training=True),
                              steps_per_epoch=300,
                              epochs=10,
                              validation_data = image_data_generator( X_valid, y_valid, batch_size=100, is_training=False),
                              validation_steps=200,
                              verbose=1,
                              shuffle=1,
                              callbacks=[checkpoint_callback])
# always save model output as soon as model finishes training
model.save(os.path.join(model_output_dir,'lane_navigation_final.h5'))